In [ ]:
#@markdown # **Latent Blending Repository Download**
#@markdown ----
!git clone https://github.com/lunarring/latentblending.git
%cd latentblending

/content/latentblending


In [ ]:
#@markdown # **Install Dependencies**
#@markdown ----
!pip install -r requirements.txt --quiet
!pip install -e . --quiet

In [ ]:
#@markdown # **Checkpoint Downloader**
#@markdown ----
model_download='https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt' #@param {type:"string"}

!wget $model_download

In [ ]:
import os, sys
import torch
torch.backends.cudnn.benchmark = False
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import warnings
import torch
from tqdm.auto import tqdm
from PIL import Image
# import matplotlib.pyplot as plt
import torch
from movie_util import MovieSaver
from typing import Callable, List, Optional, Union
from latent_blending import LatentBlending, add_frames_linear_interp
from stable_diffusion_holder import StableDiffusionHolder
import os
from tqdm import tqdm
from IPython.display import HTML
from base64 import b64encode
import gc
import torch

torch.set_grad_enabled(False)
gc.collect()
torch.cuda.empty_cache()

#@markdown # **Checkpoint Configuration**
#@markdown ----

fp_ckpt = '/content/latentblending/v2-1_768-ema-pruned.ckpt' #@param {type:"string"} 
fp_config = '/content/latentblending/configs/v2-inference-v.yaml' #@param {type:"string"} 

sdh = StableDiffusionHolder(fp_ckpt=fp_ckpt, fp_config=fp_config) 
lb = LatentBlending(sdh)


LatentDiffusion: Running in v-prediction mode
DiffusionWrapper has 865.91 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Downloading:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

autosetup_branching: num_inference_steps: 15 list_nmb_branches: [2, 3, 6] list_injection_idx: [0, 9, 12]


In [ ]:
#@markdown # **Parameter Configuration**
#@markdown ----
quality = 'low' #@param {type:"string"}
depth_strength = 0.6 #@param {type:"number"}
seed1 = 235235235 #@param {type:"number"}
seed2 = 23423526 #@param {type:"number"}
fixed_seeds = [seed1, seed2] 

prompt1 = "a beautiful painting of a cyberpunk city by sergey kolesov and pascal blanche and greg rutkowski and sachin teng. in style of digital art. colorful comic, symmetry, hyper detailed. octane render. trending on artstation " #@param {type:"string"} 

prompt2 = "a beautiful portrait painting of a cyberpunk armor by simon stalenhag and pascal blanche and alphonse mucha and nekro. in style of digital art. colorful comic, film noirs, symmetry, brush stroke, vibrating colors, hyper detailed. octane render. trending on artstation " #@param {type:"string"} 

# In seconds
duration_transition = 12 #@param {type:"number"} 
 
fps = 30 #@param {type:"number"}

lb.load_branching_profile(quality=quality, depth_strength=depth_strength)
lb.set_prompt1(prompt1)
lb.set_prompt2(prompt2)

imgs_transition = lb.run_transition(fixed_seeds=fixed_seeds)

imgs_transition_ext = add_frames_linear_interp(imgs_transition, duration_transition, fps)

# Save as MP4
fp_movie = "movie_example1.mp4"
if os.path.isfile(fp_movie):
    os.remove(fp_movie)
ms = MovieSaver(fp_movie, fps=fps, shape_hw=[sdh.height, sdh.width])
for img in tqdm(imgs_transition_ext):
    ms.write_frame(img)
ms.finalize()

def visualize_video_colab(video_path):
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)



visualize_video_colab(fp_movie)



autosetup_branching: num_inference_steps: 15 list_nmb_branches: [2, 3, 6] list_injection_idx: [0, 9, 12]


computing transition:   0%|          | 0/11 [00:00<?, ?it/s]

Initialization done. Movie shape: (768, 768, 3)
MovieSaver initialized. fps=30 crf=24 pix_fmt=yuv420p codec=libx264 preset=fast


100%|██████████| 360/360 [00:04<00:00, 83.42it/s]


Movie saved, 12s playtime, watch here: 
movie_example1.mp4
